<a href="https://colab.research.google.com/github/yemnaing/2005-stock-markets-analytics-zoomcamp/blob/main/HW1_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd

# Load the Wikipedia table of S&P 500 companies
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

# The first table contains the list of S&P 500 companies
sp500_df = tables[0]

# Select relevant columns
df = sp500_df[['Symbol', 'Security', 'Date added']].copy()

# Convert 'Date added' to datetime
df['Date added'] = pd.to_datetime(df['Date added'], errors='coerce')

# Extract the year from the 'Date added'
df['Year added'] = df['Date added'].dt.year

# Count how many stocks were added per year, excluding 1957
year_counts = df[df['Year added'] != 1957].groupby('Year added').size()

# Find the year with the highest number of additions (most recent if multiple)
max_additions = year_counts.max()
years_with_max = year_counts[year_counts == max_additions].index
most_recent_year = years_with_max.max()

print(f"Year with the highest number of additions: {most_recent_year} ({max_additions} companies added)")

# Calculate how many stocks have been in the index for more than 20 years
# (Added before or in 2005, assuming current year is 2025)
num_old_stocks = df[df['Year added'] <= 2005].shape[0]
print(f"Number of current S&P 500 stocks in the index for more than 20 years: {num_old_stocks}")


Year with the highest number of additions: 2017 (23 companies added)
Number of current S&P 500 stocks in the index for more than 20 years: 226


In [6]:
import yfinance as yf
import pandas as pd

tickers = {
    'S&P 500 (US)': '^GSPC',
    'Shanghai Composite (China)': '000001.SS',
    'HANG SENG INDEX (Hong Kong)': '^HSI',
    'S&P/ASX 200 (Australia)': '^AXJO',
    'Nifty 50 (India)': '^NSEI',
    'S&P/TSX Composite (Canada)': '^GSPTSE',
    'DAX (Germany)': '^GDAXI',
    'FTSE 100 (UK)': '^FTSE',
    'Nikkei 225 (Japan)': '^N225',
    'IPC Mexico': '^MXX',
    'Ibovespa (Brazil)': '^BVSP'
}

start_date = '2025-01-01'
end_date = '2025-05-01'

returns = {}

for name, ticker in tickers.items():
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    if data.empty:
        print(f"No data for {name} ({ticker})")
        continue
    start_price = data['Close'].iloc[0]
    end_price = data['Close'].iloc[-1]
    ytd_return = (end_price - start_price) / start_price
    returns[name] = ytd_return

returns_df = pd.DataFrame.from_dict(returns, orient='index', columns=['YTD Return'])

us_return = returns.get('S&P 500 (US)', None)

# Debug print
print(f"Type of us_return: {type(us_return)}")
print(f"Value of us_return: {us_return}")

if us_return is not None:
    # Defensive: make sure scalar (if it is a Series, get the first element)
    if isinstance(us_return, pd.Series):
        us_return = us_return.iloc[0]

    better_than_us = returns_df[returns_df['YTD Return'] > us_return]
    num_better = better_than_us.shape[0]
else:
    num_better = 0
    print("US S&P 500 return not found.")

print(f"Number of indexes with better YTD return than US S&P 500: {num_better} out of {len(tickers)}")
print(returns_df.sort_values(by='YTD Return', ascending=False))


Type of us_return: <class 'pandas.core.series.Series'>
Value of us_return: Ticker
^GSPC   -0.051033
dtype: float64
Number of indexes with better YTD return than US S&P 500: 0 out of 11
Empty DataFrame
Columns: [YTD Return]
Index: []


In [8]:
import yfinance as yf
import pandas as pd

ticker = '^GSPC'

# Use auto_adjust=True, then 'Close' is adjusted close price
data = yf.download(ticker, start='1950-01-01', progress=False, auto_adjust=True)

prices = data['Close']

all_time_highs = prices[prices.cummax() == prices].index

corrections = []

for i in range(len(all_time_highs) - 1):
    start = all_time_highs[i]
    end = all_time_highs[i + 1]
    period = prices.loc[start:end]
    low_price = period.min()
    high_price = prices.loc[start]
    drawdown_pct = (high_price - low_price) / high_price * 100
    duration = (end - start).days
    if drawdown_pct >= 5:
        corrections.append({'start': start, 'end': end, 'drawdown_pct': drawdown_pct, 'duration_days': duration})

corrections_df = pd.DataFrame(corrections)

percentiles = corrections_df['duration_days'].quantile([0.25, 0.5, 0.75])

print("Market corrections durations (days) percentiles:")
print(f"25th percentile: {percentiles.loc[0.25]:.0f}")
print(f"Median (50th percentile): {percentiles.loc[0.5]:.0f}")
print(f"75th percentile: {percentiles.loc[0.75]:.0f}")


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().